In [ ]:
!pip install face_recognition
!pip install tensorflow
!pip3 install retina-face

In [ ]:
# !git clone https://github.com/NVIDIA/apex && cd apex
# !python /content/apex/setup.py install
!apt install --allow-change-held-packages libcudnn8=8.1.0.77-1+cuda11.2

In [ ]:
!gdown --id 1-GY1pmK2hP5GGWZJQsj59kJ33tnnjdSM
!unzip /content/drive/MyDrive/liveliness/dataset.zip -d /content/

In [ ]:
!cp -r /content/drive/MyDrive/liveliness/dataset_faces /content/

In [ ]:
!mv /content/drive/MyDrive/liveliness/real/* /content/drive/MyDrive/liveliness/dataset_faces/real/

In [ ]:
%cd /content/
# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")
# import the necessary packages
from imutils import paths
import matplotlib.pyplot as plt
from pathlib import Path
import numpy as np
import argparse
import pickle
import cv2
import os
from tqdm import tqdm
# import face_recognition as rec
# from retinaface import RetinaFace

# import the necessary packages
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import RandomFlip
from tensorflow.keras.layers import RandomRotation
from tensorflow.keras.layers import RandomZoom
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import get_file
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras.layers import Rescaling
from tensorflow.keras import backend as K

/content


In [ ]:
img_height = 32
img_width = 32

# data_augmentation = Sequential(
#   [
#     RandomFlip("horizontal",
#                       input_shape=(img_height,
#                                   img_width,
#                                   3)),
#     RandomRotation(0.1),
#     RandomZoom(0.15),
#   ]
# )

class LivenessNet:
	@staticmethod
	def build(width, height, depth, classes):
		# initialize the model along with the input shape to be
		# "channels last" and the channels dimension itself
		model = Sequential()
		inputShape = (height, width, depth)
		chanDim = -1
		# if we are using "channels first", update the input shape
		# and channels dimension
		if K.image_data_format() == "channels_first":
			inputShape = (depth, height, width)
			chanDim = 1

		#adding data augmentation
		# model.add(data_augmentation)
		    
    # first CONV => RELU => CONV => RELU => POOL layer set
		model.add(Conv2D(16, (3, 3), padding="same", input_shape=inputShape))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(Conv2D(16, (3, 3), padding="same"))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(MaxPooling2D(pool_size=(2, 2)))
		model.add(Dropout(0.25))

		# second CONV => RELU => CONV => RELU => POOL layer set
		model.add(Conv2D(32, (3, 3), padding="same"))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(Conv2D(32, (3, 3), padding="same"))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(MaxPooling2D(pool_size=(2, 2)))
		model.add(Dropout(0.2))

        # first (and only) set of FC => RELU layers
		model.add(Flatten())
		model.add(Dense(64))
		model.add(Activation("relu"))
		model.add(BatchNormalization())
		model.add(Dropout(0.5))
		# softmax classifier
		model.add(Dense(classes))
		model.add(Activation("softmax"))
		# return the constructed network architecture
		return model

Attemp to pick images from directory itself

In [ ]:
# construct the argument parser and parse the arguments
num = "44"

# initialize the initial learning rate, batch size, and number of
# epochs to train for
batch_size = 64
BS = 16
EPOCHS = 400
INIT_LR = 1e-4
DEFAULT_CONFIDENCE = 0.5
# grab the list of images in our dataset directory, then initialize
# the list of data (i.e., images) and class images
print("[INFO] loading images...")

AUTOTUNE = tf.data.AUTOTUNE

data_dir = Path("/content/drive/MyDrive/liveliness/dataset_faces/")
!rm -rf /content/drive/MyDrive/liveliness/dataset_faces/.ipynb_checkpoints
image_count = len(list(data_dir.glob('*/*.jpg')))
list_ds = tf.data.Dataset.list_files(str(data_dir/'*/*'), shuffle=False)
list_ds = list_ds.shuffle(image_count, reshuffle_each_iteration=False)
val_size = int(image_count * 0.2)

train_ds = image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

class_names = train_ds.class_names

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

normalization_layer = Rescaling(1./255)
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))

num_classes = len(class_names)
print("Class names are: ", class_names)

# construct the training image generator for data augmentation
aug = ImageDataGenerator(rotation_range=20, zoom_range=0.15,
	width_shift_range=0.2, height_shift_range=0.2, shear_range=0.15,
	horizontal_flip=True, fill_mode="nearest")

# initialize the optimizer and model
print("[INFO] compiling model...")
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model = LivenessNet.build(width=32, height=32, depth=3, classes=num_classes)

# model = load_model("liveness.model29")
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), optimizer=opt, metrics=["accuracy"])

steps = len(train_ds) // BS

history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=EPOCHS
)


# evaluate the network
print("[INFO] evaluating network...")
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(EPOCHS)

#plotting and saving results
# plot the training loss and accuracy
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, EPOCHS), loss, label="train_loss")
plt.plot(np.arange(0, EPOCHS), val_loss, label="val_loss")
plt.plot(np.arange(0, EPOCHS), acc, label="train_acc")
plt.plot(np.arange(0, EPOCHS), val_acc, label="val_acc")
plt.title("Training Loss and Accuracy on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig("/content/result/plot" + num + ".png")

#saving model
print("[INFO] serializing network to '{}'...".format("liveness.model" + num))
model.save("/content/result/liveness.model" + num, save_format="h5")

USING OLD TYPE OF TRAININg TECHNIQUE AS PER PYIMAGESEARCH

In [ ]:
!cp -r /content/drive/MyDrive/liveliness/dataset_faces/ /content/

In [ ]:
num = "51"

# initialize the initial learning rate, batch size, and number of
# epochs to train for
INIT_LR = 1e-4
BS = 192
EPOCHS = 1250
DEFAULT_CONFIDENCE = 0.5
# grab the list of images in our dataset directory, then initialize
# the list of data (i.e., images) and class images
print("[INFO] loading images...")
imagePaths = list(paths.list_images("/content/dataset_faces"))
data = []
labels = []
# loop over all image paths
for imagePath in tqdm(imagePaths):
	#grab face ROI
	label = imagePath.split(os.path.sep)[-2]
	face = cv2.imread(imagePath)

	# update the data and labels lists, respectively
	shape = face.shape
	if (shape[0] > 0 and shape[1] > 0):
		data.append(face)
		labels.append(label)
    
# convert the data into a NumPy array, then preprocess it by scaling
# all pixel intensities to the range [0, 1]
data = np.array(data, dtype="float") / 255.0

# encode the labels (which are currently strings) as integers and then
# one-hot encode them
le = LabelEncoder()
labels = le.fit_transform(labels)
labels = to_categorical(labels, 2)

# partition the data into training and testing splits using 75% of
# the data for training and the remaining 25% for testing
(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.25, random_state=42)

# construct the training image generator for data augmentation
aug = ImageDataGenerator(rotation_range=20, zoom_range=0.15,
	width_shift_range=0.2, height_shift_range=0.2, shear_range=0.15,
	horizontal_flip=True, fill_mode="nearest")

# initialize the optimizer and model
print("[INFO] compiling model...")
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model = LivenessNet.build(width=32, height=32, depth=3,
	classes=len(le.classes_))

model.compile(loss="binary_crossentropy", optimizer=opt,
	metrics=["accuracy"])

# train the network
print("[INFO] training network for {} epochs...".format(EPOCHS))
H = model.fit(x=aug.flow(trainX, trainY, batch_size=BS),
	validation_data=(testX, testY), steps_per_epoch=len(trainX) // BS,
	epochs=EPOCHS)

# evaluate the network
print("[INFO] evaluating network...")
predictions = model.predict(x=testX, batch_size=BS)
print(classification_report(testY.argmax(axis=1),
	predictions.argmax(axis=1), target_names=le.classes_))

# save the network to disk
print("[INFO] serializing network to '{}'...".format("liveness.model" + num))
model.save("/content/result/liveness.model" + num, save_format="h5")
# save the label encoder to disk
f = open("/content/result/le" + num + ".pickle", "wb")
f.write(pickle.dumps(le))
f.close()

# plot the training loss and accuracy
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, EPOCHS), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, EPOCHS), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, EPOCHS), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, EPOCHS), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig("/content/result/plot" + num + ".png")


Moving results to drive

In [ ]:
# %cd /content/drive/MyDrive/liveliness/semiset
# !zip -F semiSplit.zip --out semiset.zip
# !unzip semiset.zip -d /content/drive/MyDrive/liveliness/

In [ ]:
!cp -r result /content/drive/MyDrive/liveliness/results/result51

In [ ]:
!rm -rf result/
!mkdir result

Code to cut and save faces


In [ ]:
# load our serialized face detector from disk
print("[INFO] loading face detector...")
protoPath = "/content/drive/MyDrive/liveliness/face_detector/deploy.prototxt"
modelPath = "/content/drive/MyDrive/liveliness/face_detector/res10_300x300_ssd_iter_140000.caffemodel"
# protoPath = "/Users/ruchit/Imperial/livelinessDetection/ann_approach/livenet/livenessnet/face_detector/deploy.prototxt"
# modelPath = "/Users/ruchit/Imperial/livelinessDetection/ann_approach/livenet/livenessnet/face_detector/res10_300x300_ssd_iter_140000.caffemodel"
net = cv2.dnn.readNetFromCaffe(protoPath, modelPath)

# grab the list of images in our dataset directory, then initialize
# the list of data (i.e., images) and class images
print("[INFO] loading images...")
imagePaths = list(paths.list_images("/content/semiset/jpg"))
data = []
labels = []
# loop over all image paths
for imagePath in tqdm(imagePaths):
	# extract the class label from the filename, load the image and
	# resize it to be a fixed 32x32 pixels, ignoring aspect ratio
	split = imagePath.split(os.path.sep)
	label = split[-2]
	image = cv2.imread(imagePath)

	# grab the frame dimensions and construct a blob from the frame
	(h, w) = image.shape[:2]
	#TODO: make 224
	blob = cv2.dnn.blobFromImage(cv2.resize(image, (224, 224)), 1.0, (300, 300), (104.0, 177.0, 123.0))

	# pass the blob through the network and obtain the detections and
	# predictions
	net.setInput(blob)
	detections = net.forward()

	# ensure at least one face was found
	if len(detections) > 0:
		# we're making the assumption that each image has only ONE
		# face, so find the bounding box with the largest probability
		i = np.argmax(detections[0, 0, :, 2])
		confidence = detections[0, 0, i, 2]

		# ensure that the detection with the largest probability also
		# means our minimum probability test (thus helping filter out
		# weak detections
		if confidence > DEFAULT_CONFIDENCE:
    		# compute the (x, y)-coordinates of the bounding box for
			# the face and extract the face ROI
			box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
			(startX, startY, endX, endY) = box.astype("int")
			if (endX > w or endY > h):
				img = rec.load_image_file(imagePath)
				face_locations = rec.face_locations(img)

				if (len(face_locations) == 0):
					print("out of bounds by first, no detections found by second detector")
					continue
    	    #applying third detector
					resp = RetinaFace.detect_faces(image)
					if (len(resp) == 0):
						print("out of bounds by first, no detections found by other detectors")
						continue  
					conf = 0
					faceRect = []
					for item in resp:
						response = resp[item]
						if (response['score'] > conf):
							conf = response['score']
							faceRect = response['facial_area']
					if (conf > DEFAULT_CONFIDENCE):
						startX, startY, endX, endY = faceRect
					else: 
						print("out of bounds by first, no detections found by other detectors")
						continue  
				else:
					startY, endX, endY, startX = face_locations[0]
		else:
			# confidence was low
			img = rec.load_image_file(imagePath)
			face_locations = rec.face_locations(img)
			if (len(face_locations) == 0):
				print("out of bounds by first, no detections found by other detectors")
				continue
				#applying third detector
				resp = RetinaFace.detect_faces(image)
				if (len(resp) == 0):
					print("out of bounds by first, no detections found by other detectors")
					continue  
				conf = 0
				faceRect = []
				for item in resp:
					response = resp[item]
					if (response['score'] > conf):
						conf = response['score']
						faceRect = response['facial_area']

				if (conf > DEFAULT_CONFIDENCE):
					startX, startY, endX, endY = faceRect
				else:
					print("out of bounds by first, no detections found by other detectors")
					continue
			else:
				startY, endX, endY, startX = face_locations[0]
	else:
		#no detection by first
		img = rec.load_image_file(imagePath)
		face_locations = rec.face_locations(img)
		if (len(face_locations) == 0):
			print("no detections")
			continue
			#applying third detector
			resp = RetinaFace.detect_faces(image)
			if (len(resp) == 0):
				print("no detections")
				continue  
			conf = 0
			faceRect = []
			for item in resp:
				response = resp[item]
				if (response['score'] > conf):
					conf = response['score']
					faceRect = response['facial_area']

			if (conf > DEFAULT_CONFIDENCE):
				startX, startY, endX, endY = faceRect
			else:
				print("no detections")
				continue
		else:
			startY, endX, endY, startX = face_locations[0]

	#grab face ROI
	face = image[startY:endY, startX:endX]
	
	# update the data and labels lists, respectively
	shape = face.shape
	if (shape[0] > 0 and shape[1] > 0):
		face = cv2.resize(face, (32,32))
		if(label == "real"):
			cv2.imwrite("/content/drive/MyDrive/liveliness/semiset_faces/jpg/real/" + split[-1], face)
		else:
			cv2.imwrite("/content/drive/MyDrive/liveliness/semiset_faces/jpg/fake/" + split[-1], face)
		# data.append(face)
		labels.append(label)

file = open("/content/drive/MyDrive/liveliness/semiset_faces/jpg/labels.txt", "w")
file.writelines(labels)
file.close()

[INFO] loading face detector...
[INFO] loading images...


  9%|▊         | 87/1002 [00:31<10:09,  1.50it/s]

out of bounds by first, no detections found by second detector


 23%|██▎       | 228/1002 [01:57<05:31,  2.33it/s]

out of bounds by first, no detections found by second detector


...

 83%|████████▎ | 831/1002 [04:56<00:36,  4.66it/s]

out of bounds by first, no detections found by second detector


 92%|█████████▏| 926/1002 [05:11<00:14,  5.24it/s]

out of bounds by first, no detections found by second detector


100%|██████████| 1002/1002 [05:22<00:00,  3.11it/s]
